In [1]:
from new_implement_linear import *
import matplotlib.pyplot as plt

train_path="train.csv"
test_path="test.csv"
name="discrimination_submission.csv"

In [2]:
yb_train,data_train,ids_train=load_csv_data(train_path, sub_sample=False)
#yb_test,data_test,ids_test=load_csv_data(test_path, sub_sample=False)

In [8]:
#  主函数的主体在这里
    
def main(data_train,yb_train, opt_model, cros_vali, degree,lambda_,gamma):
    seed = np.random.randint(cros_vali*100)
    data_train_copy = data_train.copy()
    print('Started the run!\n')
    
#     yb_train,data_train,ids_train=load_csv_data(train_path, sub_sample=False)
    #yb_test,data_test,ids_test=load_csv_data(test_path, sub_sample=False)
    
    data_train_copy = data_train.copy()
    print('Finished loading data!\n')   
    

    print('Feature restore...')
   
    # data_train,u_s,u_b=restore_train_first_or_none_and_second(yb_train,data_train,train_size,1)
    #data_train=restore_train_initial(yb_train,data_train,train_size)
   
    # data_train=restore_test_first_or_none(yb_train,data_train,train_size)
    # data_train=restore_test_all_b_s(yb_train,data_train,train_size,u_s)
    
    print('Feature processing...\n')
    # mean_x = np.mean(data_train,0)
    # data_train = data_train - mean_x
    #std_x = np.std(data_train,0)
    # data_train = data_train / std_x
    
    data_train = add_feature(data_train)
    
    
    #data_train=drop_feature(data_train)
    #degree=3
    #a22_value=0;
    #data_train=drop_feature(data_train)
    #yb_train,data_train=drop_22(a22_value,yb_train,data_train)
    #drop_22(2,yb_test,data_test)
    #add_feature(data_train)
    #add_feature(data_test)
    
    data_train = build_model_data(data_train, degree)
    
    #drop_feature(data_test)    
    print('Training begin...\n')
    
    print('Cross validation begins...\n')
    indices = build_k_indices(yb_train, cros_vali, seed) 
    corrections = cross_validation(data_train, yb_train, indices, opt_model)
    
    print("corrections")
    print(corrections)
    
    #data_train=destroy(yb_train,data_train)
    
    #yb_initial=yb_test.copy()
#     sign1=yb_pred+yb_id
#     sign2=yb_pred-yb_id
#     ratio=len(sign1[sign1==2])/len(sign2[sign2==2])
    
    return 
    

In [4]:
# unified model choose
def train_mode(train_x, train_y, chosen_model):
    if chosen_model   == 1:     
        w,loss=train_normal(train_y,train_x)
    
    elif chosen_model == 2:
        w,loss=train_ridge(train_y, train_x, lambda_)
    
    elif chosen_model == 3:
        w,loss=train_gradient(train_y, train_x)
    
    elif chosen_model == 4:
        w,loss=train_logistic(train_y, train_x, gamma)
        
    elif chosen_model == 5:
        w,loss=train_gradient_SGD(train_y, train_x)
                                   
#     elif chosen_model == 6:
        
    elif chosen_model == 7:
        class_1 = train_x[train_y == -1][:,1:train_x.shape[1]]
        class_2 = train_x[train_y == +1][:,1:train_x.shape[1]]
        
        w, loss = Fisher_classifier(class_1, class_2)
        
    return w, loss


# optimization models
def train_normal(yb_train, data_train):
    w,loss=least_squares(yb_train,data_train)
    return w,loss

def train_ridge(yb_train, data_train, lambda_):
    w,loss=ridge_regression(yb_train, data_train, lambda_)
    return w,loss

def train_gradient(yb_train, data_train, step_size):
    max_iters = 100
    gamma = step_size
    initial_w=np.zeros(len(data_train.T));
    w,loss=least_squares_GD(yb_train, data_train, initial_w, max_iters, gamma)
    return w,loss

def train_logistic(yb_train, data_train, step_size):
    initial_w=np.ones(len(data_train.T));
    max_iters=300;
    gamma=step_size;
    w,loss=logistic_regression(yb_train, data_train, initial_w, max_iters, gamma)
    return w,loss

def train_gradient_SGD(yb_train, data_train, step_size):
    batch_size=1;
    max_iters = 50
    gamma = step_size
    initial_w=np.zeros(len(data_train.T));
    w,loss=least_squares_SGD(yb_train, data_train, initial_w, max_iters, gamma,batch_size)
    return w,loss

# reserved def
 

def Fisher_classifier(class_1, class_2):
    # means
    m_1 = np.mean(class_1, axis = 0)
    m_2 = np.mean(class_2, axis = 0)
    
    # inner class1 dispersion
    S_1 = (class_1 - m_1).T.dot(class_1 - m_1)
    # inner class2 dispersion
    S_2 = (class_2 - m_2).T.dot(class_2 - m_2)
    # seperating level
    S_W = S_1 + S_2
    
    # perpendicular vector of the separating hyperplane
    w_star = np.linalg.inv(S_W.astype(float)).dot(m_2 - m_1)
    
    mapped_b = class_1.dot(w_star)
    m_t_b = mapped_b.mean()
    mapped_s = class_2.dot(w_star)
    m_t_s = mapped_s.mean()

    # intersection point of the separating hyperplane and the perpendicular vector 
    w0 = -0.5*(m_t_b + m_t_s)
    
    return np.insert(w_star, 0, w0), np.nan

##### Data wrangling

In [5]:
#  具体的子函数，很多现在没有用了

def test(yb_train,data_train,w,train_size,data_train_copy):
    yb_train=yb_train[train_size:]
    data_train=data_train[train_size:]
    data_train_copy=data_train_copy[train_size:]
    #y_pred=predict_labels(w, data_train)
    
    score = np.dot(data_train, w)
    y_pred=score.copy()
    y_pred[np.where(y_pred <= 0)] = -1
    y_pred[np.where(y_pred > 0)] = 1
    y_pred[np.where(data_train_copy[:,0]==-999)]=-1
    recall,precision,accuracy=performance(y_pred,yb_train)
    print("recall")
    print(recall)
    print("precision")
    print(precision)
    print("accuracy")
    print(accuracy)
    print("s%")
    print(33*recall*precision)
    return y_pred,yb_train

def test_sub(yb_test,data_test,w,ids_test):
    y_pred=predict_labels(w, data_test)
    create_csv_submission(ids_test, y_pred, name)
    recall,precision,accuracy=performance(y_pred,yb_test)
    print("recall")
    print(recall)
    
    
def restore_train_first_or_none_and_second(yb,data,train_size,maxiter):
    yb_train=yb[:train_size]
    data_train=data[:train_size]
   # yb_train=yb;
   # data_train=data;
    
    data_s=data_train[yb_train==1]
    data_b=data_train[yb_train==-1]
    
    u_s=np.zeros(len(data_s))
    u_b=np.zeros(len(data_b))
    for i in range(len(data_train.T)):
        slice_s=data_s[:,i]
        u_s[i]=np.mean(slice_s[slice_s!=-999])
        slice_s[np.where(slice_s==-999)]=u_s[i]
        
        slice_b=data_b[:,i]
        u_b[i]=np.mean(slice_b[slice_b!=-999])
        slice_b[np.where(slice_b==-999)]=u_b[i]
        
    data_train[yb_train==1]=data_s
    data_train[yb_train==-1]=data_b
    #print("iter_=");
    #print(iter_);
    #print(np.abs(u_s_new-u_b_new)/u_s_new);
    return data,u_s,u_b

def restore_train_initial(yb,data,train_size):

    yb_train=yb[train_size:]
    data_train=data[train_size:]
    
    maxiter=1;
    iter_=0;
    u_old=np.ones(len(data_train.T))*-999
    err=10;
    #data_short=
    u=np.mean(data_train,0)
    #data_s=data_train[yb_train==1]
    #data_b=data_train[yb_train==-1]
    #u_s=np.mean(data_s,0)
    #u_b=np.mean(data_b,0)
    
    while iter_<maxiter and err>1e-2:
        u_new=np.mean(data_train,0)
        for i in range(len(data_train.T)):
            slice_=data_train[:,i]
            slice_[np.where(slice_==u_old[i])]=u_new[i]
        err=np.linalg.norm(np.linalg.norm(np.abs(u_new-u_old)/u_new))
        u_old=u_new
        iter_+=1
    #print("iter_=");
    #print(iter_);
   # print(np.abs(u_new-u)/u_new);
    return data

def restore_test_first_or_none(yb,data,train_size):

    yb_test=yb[train_size:]
    data_test=data[train_size:]
    #data_s=data_train[yb_train==1]
    #data_b=data_train[yb_train==-1]
    #u_s=np.mean(data_s,0)
    #u_b=np.mean(data_b,0)
    for i in range(len(data_test.T)):
        #slice_=data_s[:,i]
        #slice_[np.where(slice_==-999)]=u_s[i]
        #slice_=data_b[:,i]
        #slice_[np.where(slice_==-999)]=u_b[i]
        slice_=data_test[:,i]
        
        u=np.mean(slice_[slice_!=-999])
        slice_[np.where(slice_==-999)]=u
    #data_train[yb==1]=data_s
    #data_train[yb==-1]=data_b
    return data

def restore_test_second(yb,data,train_size,u_s,u_b):
    
    yb_test=yb;
    #yb_test=yb[train_size:]
    data_test=data[train_size:]
    
    data_s=data_test[yb_test==1]
    data_b=data_test[yb_test==-1]
    for i in range(len(data_test.T)):
        #slice_=data_s[:,i]
        #slice_[np.where(slice_==-999)]=u_s[i]
        #slice_=data_b[:,i]
        #slice_[np.where(slice_==-999)]=u_b[i]
        slice_s=data_s[:,i]
        slice_s[np.where(slice_s==-999)]=u_s[i]
        
        slice_b=data_b[:,i]
        slice_b[np.where(slice_b==-999)]=u_b[i]
    data_test[yb_test==1]=data_s
    data_test[yb_test==-1]=data_b
    return data

def restore_test_all_b_s(yb,data,train_size,value):

    yb_test=yb[train_size:]
    data_test=data[train_size:]
    #data_s=data_train[yb_train==1]
    #data_b=data_train[yb_train==-1]
    #u_s=np.mean(data_s,0)
    #u_b=np.mean(data_b,0)
    for i in range(len(data_test.T)):
        slice_=data_test[:,i]
        slice_[np.where(slice_==-999)]=value[i]
    #data_train[yb==1]=data_s
    #data_train[yb==-1]=data_b
    return data

def drop_feature(data):

    #data=np.delete(data,range(23,29),1)
    data=np.delete(data,20,1)
    data=np.delete(data,range(17,18),1)    
    data=np.delete(data,range(15,17),1)
    #data=np.delete(data,12,1)
    data=np.delete(data,7,1)   
    data=np.delete(data,3,1) 
    #data=np.delete(data,range(4,7),1)
    
#r_data=np.delete(r_data,17,1)
#r_data=np.delete(r_data,18,1)


#r_data=np.delete(r_data,25,1)
#r_data=np.delete(r_data,25,1)
#r_data=np.delete(r_data,28,1)
#r_data=np.delete(r_data,29,1)
    return data

def drop_22(a22_value,yb,data):
    data_=data[data[:,22]==a22_value]
    yb_=yb[data[:,22]==a22_value]
    data_=np.delete(data_,22,1)
    return yb_,data_

def add_feature(data):
    r_data=data.copy()#np.c_[tx,x**(i+1)]
    r_data=np.c_[r_data,np.exp(data[:,4])]
    r_data=np.c_[r_data,np.exp(data[:,6])]
    r_data=np.c_[r_data,np.exp(data[:,7])]
    #r_data=np.c_[r_data,np.exp(data[:,11])]
    r_data=np.c_[r_data,np.exp(data[:,12])]
    r_data=np.c_[r_data,np.exp(data[:,14])]
    r_data=np.c_[r_data,np.exp(data[:,15])]
    r_data=np.c_[r_data,np.exp(data[:,24])]
    r_data=np.c_[r_data,np.exp(data[:,18])]
    #r_data=np.c_[r_data,np.exp(data[:,11])]
    #r_data=np.c_[r_data,np.exp(data[:,27])]
    #r_data=np.c_[r_data,np.exp(data[:,27])]

    r_data=np.c_[r_data,np.cos(data[:,10])]
    r_data=np.c_[r_data,np.cos(data[:,24])]
    r_data=np.c_[r_data,np.cos(data[:,27])]

    r_data=np.c_[r_data,np.cos(data[:,11])]
    r_data=np.c_[r_data,np.log(data[:,9])]

    r_data=np.c_[r_data,np.log(data[:,10])]
    r_data=np.c_[r_data,np.log(data[:,13])]
    r_data=np.c_[r_data,np.log(data[:,19])]

    #r_data=np.c_[r_data,data[:,3]*data[:,9]]
    #r_data=np.c_[r_data,data[:,2]*data[:,19]]
    #r_data=np.c_[r_data,data[:,22]*data[:,12]]
    #for i in range(len(data.T)-10):
   # for j in range(i+10,len(data.T)):
     #       r_data=np.c_[r_data,data[:,i]*data[:,i+10]]
    
    r_data=np.c_[r_data,data[:,range(7,10)]**5]
#r_data=np.c_[r_data,1/data[:,9]]
#r_data=np.c_[r_data,1/data[:,26]]

#r_data=np.c_[r_data,np.log(data[:,19])]
#r_data=np.c_[r_data,np.log(data[:,20])]
#r_data=np.c_[r_data,np.sin(data[:,11])]
#r_data=np.c_[r_data,np.exp(data[:,10])]
    return r_data

##### Cross validation

In [6]:
def build_k_indices(y, k_fold, seed):
    """build k indices for k-fold."""
    num_row = y.shape[0]#注意.shape[0]
    interval = int(num_row / k_fold)
    np.random.seed(seed)
    indices = np.random.permutation(num_row)
    k_indices = [indices[k * interval: (k + 1) * interval]
                for k in range(k_fold)]
    return np.array(k_indices)


def cross_validation(feature, lable, indices, opt_model):
    '''cross validation to evaluate performance of a given mode.
    
       feature (ndarray)  : feature vectors
       lable   (array)    : lable of every row of feature
       indices            : grouped indices of the dataset
       opt_model(int)     : chosen classification model 
    '''
    
    iter_times = indices.shape[0]
    correction = []
    
    for ct_iter in range(iter_times):
        print('iteration:',ct_iter)
        # produce training data
        train_x = feature[indices[ct_iter]]
        train_y = lable[indices[ct_iter]]
        
        # produce testing data
        test_indices = indices[np.arange(indices.shape[0])!=ct_iter]
        test_indices = test_indices.ravel()
        test_x = feature[test_indices]
        test_y = lable[test_indices]
        
        w, loss = train_mode(train_x, train_y, opt_model)
        
        prediction = predict_labels(w, test_x)
            
        correction.append( sum(prediction == test_y.astype(float))/test_y.shape[0] )
        
    return correction

In [9]:
#   主函数在这里
#  opt_model=1  train_normal equation
#  opt_model=2  train_ridge regression
#  opt_model=3   其他的暂时有bug
#  ...
#  opt_model=7  Fisher Linear Discriminant

opt_model=7
cros_slice = 8
degree=1
lambda_=0    

for gamma in [1e-9,1e-5,1e-2,1,100]:#[1e-10,1e-8,1e-7,1e-6]:#[1e-7,1e-5,1e-2,1,100]:
    main(data_train, yb_train, opt_model, cros_slice, degree, lambda_, gamma)

Started the run!

Finished loading data!

Feature restore...
Feature processing...

Training begin...

Cross validation begins...

iteration: 0
iteration: 1
iteration: 2
iteration: 3
iteration: 4
iteration: 5
iteration: 6
iteration: 7
corrections
[0.751872, 0.7513051428571429, 0.7522605714285714, 0.7518902857142857, 0.7512594285714286, 0.749408, 0.7517302857142857, 0.752352]
Started the run!

Finished loading data!

Feature restore...
Feature processing...

Training begin...

Cross validation begins...

iteration: 0
iteration: 1
iteration: 2
iteration: 3
iteration: 4
iteration: 5
iteration: 6
iteration: 7
corrections
[0.7516937142857143, 0.7510765714285714, 0.751488, 0.7506651428571428, 0.7521097142857143, 0.7514102857142857, 0.7512502857142858, 0.7507245714285714]
Started the run!

Finished loading data!

Feature restore...
Feature processing...

Training begin...

Cross validation begins...

iteration: 0
iteration: 1
iteration: 2
iteration: 3
iteration: 4
iteration: 5
iteration: 6
it

KeyboardInterrupt: 